In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd
from datetime import date, timedelta
import numpy as np
from IPython import get_ipython

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float

In [5]:
base = pd.read_excel('../data/SOFR-10012019-10292020.xls',"nice format")
base['fecha_inicial']=base['DATE']
base['fecha_inicial']=pd.to_datetime(base['fecha_inicial'])
base['fecha_inicial']=base['fecha_inicial'].dt.date
base['tasa']=base['RATE\n(PERCENT)']/100
#se divide en 100 para llevarlo a decimal
alt = pd.concat([base['fecha_inicial'], base['fecha_inicial'].shift()], axis=1)
alt = alt.fillna(date(2020,10,29))
base['fecha_final'] = alt.iloc[:,1]
base['fecha_final']=pd.to_datetime(base['fecha_final'])
base['fecha_final']=base['fecha_final'].dt.date
base=base[['fecha_inicial','fecha_final','tasa']]
base

,fecha_inicial,fecha_final,tasa
0,2020-10-28,2020-10-29,0.0008
1,2020-10-27,2020-10-28,0.0009
2,2020-10-26,2020-10-27,0.0009
3,2020-10-23,2020-10-26,0.0008
4,2020-10-22,2020-10-23,0.0007
...,...,...,...
265,2019-10-07,2019-10-08,0.0183
266,2019-10-04,2019-10-07,0.0182
267,2019-10-03,2019-10-04,0.0184
268,2019-10-02,2019-10-03,0.0185


In [6]:
def get_wf(fecha_inicial: date, fecha_final:date, valor_tasa:float) -> float:
    
    qc_fec_ini = Qcf.QCDate(fecha_inicial.day, fecha_inicial.month, fecha_inicial.year)
    qc_fec_fin = Qcf.QCDate(fecha_final.day, fecha_final.month, fecha_final.year)
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    return qc_tasa.wf(qc_fec_ini, qc_fec_fin)

In [7]:
base['wf'] = base.apply(
    lambda row: get_wf(row['fecha_inicial'], row['fecha_final'], row['tasa']),
    axis=1)
fixings=base['wf']
def get_accrued_rate(start_date: Qcf.QCDate, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
    dias = start_date.day_diff(accrual_date)
    prod = base['wf'].prod()
    tasa_eq = (prod - 1) * 360.0 / dias
    return tasa_eq

start_date=Qcf.QCDate(1,10,2019)
accrual_date=Qcf.QCDate(31,10,2019)
tasa_eq=get_accrued_rate(start_date,accrual_date, fixings)
print(tasa_eq)

0.0942474893949008


In [8]:
def get_accrued_interest(nocional, tasa_eq, dias, start_date: Qcf.QCDate, accrual_date: Qcf.QCDate) -> float:
    interes_flot_ois = nocional * tasa_eq * dias / 360
    return interes_flot_ois

nocional = 100000000
start_date=Qcf.QCDate(3,12,2019)
accrual_date=Qcf.QCDate(31,12,2019)
dias = start_date.day_diff(accrual_date)
tasa_eq=get_accrued_rate(start_date,accrual_date, fixings)
interes_flot_ois=get_accrued_interest(nocional, tasa_eq, dias, start_date, accrual_date)
print(interes_flot_ois)

785395.7449575065


In [9]:
def amount(nocional, tasa_eq, start_date, accrual_date) -> float:
    qc_tasa_eq = Qcf.QCInterestRate(tasa_eq, Qcf.QCAct360(), Qcf.QCLinearWf())
    amount = nocional + nocional*(qc_tasa_eq.wf(start_date, accrual_date)-1)
    return amount
nocional = 100000000
start_date=Qcf.QCDate(3,12,2019)
accrual_date=Qcf.QCDate(31,12,2019)
amount = amount(nocional, tasa_eq, start_date, accrual_date)
print(amount)

100785395.7449575


In [10]:
get_ipython().magic('reset -sf')

AD: ¿Otro `import`? No es necesario para los paquetes que ya se importaron en la primera celda.

In [11]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd
from datetime import date, timedelta
import numpy as np
from IPython import get_ipython

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

from functools import partial

In [12]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [13]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [14]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)


In [15]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float

In [16]:
ois = OisCashflow(
    Qcf.QCDate(13, 1, 2020),
    Qcf.QCDate(13, 1, 2021),
    Qcf.QCDate(13, 1, 2021),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [17]:
fwd = Qcf.ForwardRates()
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [18]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    fwd.set_rates_icp_clp_leg(val_date, 1.0, ois_cashflow[1], zcc, fixings)
    aux.show_leg(op[1], 'OisCashflow', '').style.format(frmt)
    pass

In [19]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
df_curva.head().style.format(frmt)
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
    vp_fija = vp.pv(val_date, oiscashflow[0], zcc, fixings)
    vp_flot = vp.pv(val_date, oiscashflow[1], zcc, fixings)
    print(f'Por lo tanto, el valor total de la operación es:\nValor total: USD {vp_fija + vp_flot:,.2f}')
    pass

## Tests

In [20]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [21]:
amount_tol = 10000
rate_tol = .001

In [22]:
exitos = 0

In [23]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [25]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [26]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [27]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [28]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [29]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [30]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

AttributeError: 'OisCashflow' object has no attribute 'get_accrued_rate'

Se verificará el cálculo usando `df_fixings`.

In [31]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [32]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [33]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [34]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [35]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [36]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [37]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [38]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [39]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [40]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [41]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [42]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

NameError: name 'accrued_rate' is not defined

In [43]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

NameError: name 'accrued_rate' is not defined

### `OisCashflow.accrued_interest`

In [44]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

AttributeError: 'OisCashflow' object has no attribute 'get_accrued_interest'

In [45]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

NameError: name 'accrued_rate' is not defined

In [46]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

NameError: name 'accrued_interest' is not defined

### `OisCashflow.amount`

In [47]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

AttributeError: 'OisCashflow' object has no attribute 'amount'

In [48]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [49]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

NameError: name 'amount' is not defined

In [50]:
suma_exito(amount, check_amount, 'monto')

NameError: name 'amount' is not defined

### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [51]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [52]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [53]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [54]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [55]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: 'OisCashflow' object is not subscriptable

In [56]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: 'OisCashflow' object is not subscriptable

In [57]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: 'OisCashflow' object is not subscriptable

In [58]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: 'OisCashflow' object is not subscriptable

In [59]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: 'OisCashflow' object is not subscriptable

### `present_value`

In [60]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: 'OisCashflow' object is not subscriptable

In [61]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: 'OisCashflow' object is not subscriptable

In [ ]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

In [62]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: 'OisCashflow' object is not subscriptable

In [63]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: 'OisCashflow' object is not subscriptable

In [64]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 0
